In [3]:
from arcgis import GIS
from arcgis.geocoding import geocode
from arcgis import network

In [4]:
gis=GIS('https://arcgis.com','bhammersley_tech')

Enter password: ········


<h1> Geocoding </h1>

In [5]:
from arcgis.geocoding import geocode

result = geocode('Cambridge, UK')
result

[{'address': 'Cambridge, Cambridgeshire, England',
  'location': {'x': 0.11161000000004151, 'y': 52.20992000000007},
  'score': 100,
  'attributes': {'Loc_name': 'World',
   'Status': 'T',
   'Score': 100,
   'Match_addr': 'Cambridge, Cambridgeshire, England',
   'LongLabel': 'Cambridge, Cambridgeshire, England, GBR',
   'ShortLabel': 'Cambridge',
   'Addr_type': 'Locality',
   'Type': 'City',
   'PlaceName': 'Cambridge',
   'Place_addr': 'Cambridgeshire, England',
   'Phone': '',
   'URL': '',
   'Rank': 5.5,
   'AddBldg': '',
   'AddNum': '',
   'AddNumFrom': '',
   'AddNumTo': '',
   'AddRange': '',
   'Side': '',
   'StPreDir': '',
   'StPreType': '',
   'StName': '',
   'StType': '',
   'StDir': '',
   'BldgType': '',
   'BldgName': '',
   'LevelType': '',
   'LevelName': '',
   'UnitType': '',
   'UnitName': '',
   'SubAddr': '',
   'StAddr': '',
   'Block': '',
   'Sector': '',
   'Nbrhd': '',
   'District': '',
   'City': 'Cambridge',
   'MetroArea': '',
   'Subregion': 'Cambri

In [6]:
len(result)

12

In [7]:
for i in range(len(result)):
    print("Result", i, ": Score:", result[i]['score'], "for", result[i]['attributes']['Match_addr'] )

Result 0 : Score: 100 for Cambridge, Cambridgeshire, England
Result 1 : Score: 100 for Cambridge, Gloucester, Gloucestershire, England
Result 2 : Score: 100 for Cambridge, England
Result 3 : Score: 100 for Cambridge
Result 4 : Score: 100 for Cambridge
Result 5 : Score: 100 for Cambridge
Result 6 : Score: 100 for Cambridge
Result 7 : Score: 98.73 for The Cambridge
Result 8 : Score: 98.73 for The Cambridge
Result 9 : Score: 95.6 for Cambridge North
Result 10 : Score: 95.6 for Cambridge North
Result 11 : Score: 95 for Combridge, Uttoxeter, Staffordshire, England


In [8]:
m = gis.map()
m.center = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},'x': 12424.368367441673,'y': 6838175.370353744}
m.zoom=12
m

MapView(layout=Layout(height='400px', width='100%'), zoom=12.0)

In [9]:
symbol = {
    "type" : "simple-marker",
    "style" : "diamond",
    "color" : [230,76,0,1]
    }

m.draw(result[0]['location'], symbol=symbol)

<h1> Network analysis - Find nearest neighbour</h1>

In [10]:
m = gis.map()
m.center = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},'x': 9456.60874216188,'y': 6842717.624355748}
m.zoom = 14
m

MapView(layout=Layout(height='400px', width='100%'), zoom=14.0)

In [11]:
ambulances = gis.content.get('4800f5a1dbbd4b1f8c7775aa6f4fa0f3')
incident = gis.content.get('3189390b46e44427b03a6dcbcea27875')
m.add_layer(ambulances)
m.add_layer(incident)

In [12]:
analysis_url = gis.properties.helperServices.closestFacility.url
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=gis)

In [13]:
travel_modes = cf_layer.retrieve_travel_modes()

for t in travel_modes['supportedTravelModes']:
    print(t['name'])

Walking Time
Rural Driving Distance
Driving Time
Driving Distance
Walking Distance
Rural Driving Time
Trucking Time
Trucking Distance
Cycling
Tank


In [14]:
drive_time = [t for t in travel_modes['supportedTravelModes'] if t['name']=='Driving Time'][0]

In [15]:
# Convert to featuresets

amb_lyr = ambulances.layers[0]
amb_fset=amb_lyr.query(out_sr=4326)

inc_lyr = incident.layers[0]
inc_fset = inc_lyr.query(out_sr=4326)

In [16]:
result = cf_layer.solve_closest_facility(incidents=inc_fset,
                                        facilities=amb_fset,
                                        default_target_facility_count=1,
                                        return_directions=True,
                                        return_facilities=True,
                                        travel_mode = drive_time,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])

In [17]:
result

{'messages': [{'type': 50,
   'description': 'Input field [Availability] was not mapped to a field in the network analysis class "Facilities".'},
  {'type': 50,
   'description': 'Input field [Type] was not mapped to a field in the network analysis class "Incidents".'}],
 'routes': {'fieldAliases': {'ObjectID': 'ObjectID',
   'FacilityID': 'FacilityID',
   'FacilityRank': 'FacilityRank',
   'Name': 'Name',
   'IncidentCurbApproach': 'IncidentCurbApproach',
   'FacilityCurbApproach': 'FacilityCurbApproach',
   'IncidentID': 'IncidentID',
   'StartTime': 'StartTime',
   'EndTime': 'EndTime',
   'StartTimeUTC': 'StartTimeUTC',
   'EndTimeUTC': 'EndTimeUTC',
   'Total_Minutes': 'Total_Minutes',
   'Total_TravelTime': 'Total_TravelTime',
   'Total_Miles': 'Total_Miles',
   'Total_Kilometers': 'Total_Kilometers',
   'Total_TimeAt1KPH': 'Total_TimeAt1KPH',
   'Total_WalkTime': 'Total_WalkTime',
   'Total_TruckMinutes': 'Total_TruckMinutes',
   'Total_TruckTravelTime': 'Total_TruckTravelTime',

In [18]:
amb_id = result['routes']['features'][0]['attributes']['FacilityID']
dist = result['routes']['features'][0]['attributes']['Total_Kilometers']

print('The closest ambulance to the incident has ID:'+str(amb_id)+' and is '+str(dist)+'Km from the incident')

The closest ambulance to the incident has ID:3 and is 0.3448616195742563Km from the incident


In [19]:
from arcgis.features import Feature, FeatureSet
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)
        
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])
    
allocation_line_symbol_4 = {'type': 'esriSLS', 'style': 'esriSLSSolid',
                            'color': 
                            [0,0,255,100], 'width': 6}
m.draw(routes_fset, symbol = allocation_line_symbol_4)

<h2> Return directions </h2>

In [20]:
for d in result['directions'][0]['features']:
    print(d['attributes']['text'])

Start at 1 Pepys Way, Girton, Cambridge CB3 0PA
Go north on Girton Road toward Pepys Way / Cambridge Road
Turn left on Pepys Way
Turn right on St Vincent's Close
Finish at Location 3
